In [ ]:
%matplotlib inline

import lsqfit
from model_avg_paper import *
from model_avg_paper.test_model_vary import test_vary_poly

In [ ]:
p0_test_quad = {
    'a0': 1.80,
    'a1': -0.53,
    'a2': 0.31,
}
Nt = 16
noise_params = {
    'noise_amp': 1.0,
    'noise_samples': 160,
    'frac_noise': False,
}
obs_name='a0'

def quad_model(x,p):
    return poly_model(x,p,Nt,m=2)

# Set seed for consistency of outcome
np.random.seed(96737)

In [ ]:
test_data = gen_synth_data(
    np.arange(1,Nt),
    p0_test_quad, 
    quad_model,
    **noise_params)

test_data

In [ ]:
test_res = test_vary_poly(test_data, m_max=5, m_min=0, Nt=Nt, obs_name=obs_name, IC='AIC')

In [ ]:
# Prepare table for paper

import pandas as pd

test_res_df = pd.DataFrame({ k: test_res[k] for k in ['obs', 'Qs', 'ICs', 'fits', 'probs']})
for pn in ('a1', 'a2', 'a3', 'a4', 'a5'):
    test_res_df[pn] = test_res_df['fits'].apply(lambda f: f.p.get(pn))
test_res_df['chi2'] = test_res_df['fits'].apply(lambda f: f.chi2)
test_res_df['probs'] /= np.sum(test_res_df['probs'])
tdf = test_res_df.drop('fits', axis=1).rename(columns={'obs': 'a0'})
tdf['probs'] = tdf['probs'].round(2)
tdf['chi2'] = tdf['chi2'].round(2)
tdf['Qs'] = tdf['Qs'].round(2)
tdf['ICs'] = -2*tdf['ICs'].round(2)

print(test_res['obs_avg'])

tdf

In [ ]:
## Figure 1A

plot_gvcorr(test_res['data']['y'], x=np.arange(1,Nt), color='forestgreen')

TF = test_res['fits'][2]
plot_gvcorr(TF.fcn(np.arange(0,18), p0_test_quad), 
            x=np.arange(0,18), color='k', fill=True)

plt.xlabel('$x$')
plt.ylabel('$y(x)$')
plt.xlim(0,17)

plt.tight_layout()

# Uncomment to save figure to disk
#plt.savefig('plots/poly_data.pdf', bbox_inches = "tight")

In [ ]:
## Figure 1B

import matplotlib.ticker as ticker

gs = plt.GridSpec(2, 1, height_ratios=[2,1])
gs.update(hspace=0.06)

ax1 = plt.subplot(gs[0])

plot_gvcorr([test_res['obs_avg']], x=np.array([-0.1]), color='red', markersize=9, marker='s', open_symbol=True, label='Model avg.')
plot_gvcorr(test_res['obs'], x=test_res['m'], label='Individual fits')

ax1.plot(np.array([-1,10]), 0*np.array([0,0])+p0_test_quad[obs_name], linestyle='--', color='k', label='Model truth')
ax1.set_xlabel('$m$')
ax1.set_ylabel('$a_0$')

ax1.legend(loc='center left', bbox_to_anchor=(1,0.5))
ax1.set_xlim(-0.3, 5.3)

plt.setp(ax1.get_xticklabels(), visible=False)

ax2 = plt.subplot(gs[1])
ax2.set_xlim(-0.3, 5.3)
ax2.set_ylim(0,0.65)

p_norm = test_res['probs'] / np.sum(test_res['probs'])
Q_norm = test_res['Qs'] / np.sum(test_res['Qs'])
plt.plot(test_res['m'], p_norm, color='orange', label='pr$(M|D)$')
plt.plot(test_res['m'], Q_norm, color='blue', linestyle='-.', label='Fit p-value')  # Note: fit prob != model prob! 

tick_spacing = 1
ax2.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))
plt.yticks([0,np.max(p_norm)])
ax2.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '0' if x == 0 else '{:.2f}'.format(x)))



# Put a legend to the right of the current axis
ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax2.set_xlabel('$m$')
ax2.set_ylabel('p')

# Uncomment to save figure to disk
#plt.savefig('plots/poly_avg.pdf', bbox_inches = "tight")

In [ ]:
# Scaling w/number of samples

Nsamp_array = np.array([20, 40, 80, 160, 320, 640, 2040, 4096, 4096*2, 4096*4])
Nsamp_max = Nsamp_array[-1]

noise_params['noise_samples'] = Nsamp_max
scale_data = gen_synth_data(
    np.arange(1,Nt), 
    p0_test_quad, 
    quad_model,
    **noise_params)

model_avg_vs_Nsamp = []
fixed_quad_vs_Nsamp = []
fixed_quart_vs_Nsamp = []
naive_avg_vs_Nsamp = []
fw_vs_Nsamp = []

for Nsamp in Nsamp_array:
    test_data_scale = cut_synth_data_Nsamp(scale_data, Nsamp)
    test_res_scale = test_vary_poly(test_data_scale, m_max=5, m_min=0, Nt=Nt, obs_name=obs_name, IC='AIC')
    test_res_scale_naive = test_vary_poly(test_data_scale, m_max=5, m_min=0, Nt=Nt, obs_name=obs_name, IC='naive')
    
    model_avg_vs_Nsamp.append(test_res_scale['obs_avg'])
    naive_avg_vs_Nsamp.append(test_res_scale_naive['obs_avg'])
    fixed_quad_vs_Nsamp.append(test_res_scale['obs'][2])
    fixed_quart_vs_Nsamp.append(test_res_scale['obs'][4])
    fw_vs_Nsamp.append(obs_avg_full_width(test_res_scale['obs'], test_res_scale['Qs'], test_res_scale['fits'], bf_i=2))
    

In [ ]:
## Figure 2

plot_gvcorr(model_avg_vs_Nsamp, x=np.log(Nsamp_array)+0.1, marker='o', markersize=6, label='Model avg. (AIC)')
plot_gvcorr(fixed_quad_vs_Nsamp, x=np.log(Nsamp_array)+0.2, color='red', marker='s', markersize=6,label='Quadratic fixed')
plot_gvcorr(fw_vs_Nsamp, x=np.log(Nsamp_array)+0.3, color='orange', marker='X', markersize=8,label='Full-width systematic')
plot_gvcorr(naive_avg_vs_Nsamp, x=np.log(Nsamp_array)+0.4, color='silver', marker='v', markersize=8,label='Model avg. (naive)')

plt.plot(np.array([0,10]), 0*np.array([0,10])+p0_test_quad[obs_name], linestyle='--', color='k', label='Model truth')
plt.xlabel('$\\log(N)$')
plt.ylabel('$a_0$')
plt.xlim(2.7,7.)
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left')

# Put a legend to the right of the current axis
ax = plt.subplot(111)
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

#plt.show()
#plt.ylim(0.78,0.82)
#plt.tight_layout()

# Uncomment to save figure to disk
#plt.savefig('plots/poly_N_scaling.pdf', bbox_inches = "tight")